# 🦠 Agent-Based Contagion Spread Simulation

This project presents an interactive, agent-based simulation of contagion dynamics in a confined 2D space. It models how infectious diseases can spread in a population with varying degrees of vaccination (immunization) and explores how these parameters influence epidemic outcomes over time.


# Overview

Agents (representing individuals) move in a 50×50 nm² box and can be in one of three states:

* **Healthy**
* **Infected**
* **Immunized** (vaccinated)

## Model

The model incorporates realistic dynamics:

* Infections occur through proximity-based contact.
* Immunized individuals have a reduced probability of getting infected (simulating breakthrough cases).
* Infected agents eventually recover and become immunized again, allowing for reinfection dynamics (endemic behavior).


## Features

* **Interactive simulation** using Jupyter widgets
* **Parameter controls** for population size, infection radius, probability, vaccination rate, and simulation length
* **Live visualizations**:

  * Agent positions and infection status
  * Infection trend over time
* **Modular code** structure for easy extension or integration into dashboards

<img src="contagion_demo.png" alt="Contagion Simulation Demo" style="width:40%;">

## Packages Used

* Python (NumPy, Matplotlib)
* Jupyter Notebook
* `ipywidgets` for interactivity
* Custom module (`sim_code.py`) for agent logic


# How to Run

1. Clone this repo and ensure the following files are present:

   * `dynamic_plotting.ipynb` (main notebook)
   * `sim_code.py` (simulation functions)

2. Install the required packages:

   ```bash
   pip install numpy matplotlib ipywidgets
   ```

3. Open the notebook in Jupyter:

   ```bash
   jupyter notebook dynamic_plotting.ipynb
   ```

4. Adjust parameters using sliders and run the simulation.

5. Observe the live updates of agent states and infection trends.


## Ideas for Extension

* Add real-world data integration via APIs
* Build a web app using Streamlit or Dash
* Compare policy scenarios across regions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, VBox, HBox, Play, jslink, interactive_output
from IPython.display import display

from sim_code import run_simulation


In [2]:

# Color mapping for agent states
state_colors = {0: 'blue', 1: 'red', 2: 'green'}

def plot_simulation(population, infection_radius, infection_probability, steps, 
                    box_size=50, init_infected=1, immunized_fraction=0.0, seed=None, t_show=None):
    """
    Run simulation and plot results.
    """
    history = run_simulation(
        population=population,
        infection_radius=infection_radius,
        infection_probability=infection_probability,
        steps=steps,
        box_size=box_size,
        init_infected=init_infected,
        immunized_fraction=immunized_fraction,
        seed=seed
    )
    if t_show is None:
        t_show = steps - 1

    # Scatter plot at selected time step
    agents = history[t_show]
    plt.figure(figsize=(6, 6))
    for state, color in state_colors.items():
        idx = agents[:, 2] == state
        plt.scatter(agents[idx, 0], agents[idx, 1], c=color, label={0:'Healthy',1:'Infected',2:'Immunized'}[state], s=30)
    plt.title(f"Agent States at Step {t_show+1}")
    plt.xlim(0, box_size)
    plt.ylim(0, box_size)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Line plot of infected count over time
    infected_counts = np.sum(history[:, :, 2] == 1, axis=1)
    plt.figure(figsize=(6, 3))
    plt.plot(np.arange(steps), infected_counts, color='red')
    plt.xlabel("Time step")
    plt.ylabel("Number infected")
    plt.title("Infected Over Time")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:

# --- Interactive controls ---

pop_slider = IntSlider(value=50, min=10, max=500, step=10, description='Population')
radius_slider = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Infect Radius')
prob_slider = FloatSlider(value=0.2, min=0.0, max=1.0, step=0.01, description='Infect Prob')
steps_slider = IntSlider(value=200, min=50, max=2000, step=10, description='Steps')
immun_slider = FloatSlider(value=0.0, min=0.0, max=0.9, step=0.05, description='Immunized %')
seed_slider = IntSlider(value=0, min=0, max=10000, step=1, description='Seed')


In [4]:

def interactive_sim(population, infection_radius, infection_probability, steps, immunized_fraction, seed):
    history = run_simulation(
        population=population,
        infection_radius=infection_radius,
        infection_probability=infection_probability,
        steps=steps,
        immunized_fraction=immunized_fraction,
        seed=seed
    )
    # Time slider for stepping through simulation
    t_slider = IntSlider(value=steps-1, min=0, max=steps-1, step=1, description='Time')
    play = Play(value=steps-1, min=0, max=steps-1, step=1, interval=80, description="Press play", disabled=False)
    jslink((play, 'value'), (t_slider, 'value'))

    def plot_at_time(t_show):
        agents = history[t_show]
        plt.figure(figsize=(6, 6))
        for state, color in state_colors.items():
            idx = agents[:, 2] == state
            plt.scatter(agents[idx, 0], agents[idx, 1], c=color, label={0:'Healthy',1:'Infected',2:'Immunized'}[state], s=30)
        plt.title(f"Agent States at Step {t_show+1}")
        plt.xlim(0, 50)
        plt.ylim(0, 50)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()
        plt.grid(True)
        plt.show()

        # Line plot of infected count up to current time
        infected_counts = np.sum(history[:, :, 2] == 1, axis=1)
        plt.figure(figsize=(6, 3))
        plt.plot(np.arange(t_show+1), infected_counts[:t_show+1], color='red')
        plt.xlim(0, steps)
        plt.ylim(0, np.max(infected_counts)+1)
        plt.axvline(t_show, color='gray', linestyle='--', alpha=0.5)
        plt.xlabel("Time step")
        plt.ylabel("Number infected")
        plt.title("Infected Over Time")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    out = interactive_output(plot_at_time, {'t_show': t_slider})
    display(HBox([play, t_slider]))
    display(out)


In [5]:
interact(
    interactive_sim,
    population=pop_slider,
    infection_radius=radius_slider,
    infection_probability=prob_slider,
    steps=steps_slider,
    immunized_fraction=immun_slider,
    seed=seed_slider
);

interactive(children=(IntSlider(value=50, description='Population', max=500, min=10, step=10), FloatSlider(val…

## Key Insights

This simulation helps visualize the impact of:

* Vaccination coverage on infection peaks and spread duration
* Immunity thresholds required to prevent endemic scenarios
* How changing the chance of breakthrough infections influences outcomes